# Код работы

## Загрузка модулей

In [1]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime
import locale
import pymorphy3

import src.spellcheck
from src.dircheck import get_new_file_names
from src.processor import *

In [2]:
locale.setlocale(locale.LC_ALL, 'ru_RU')

'ru_RU'

## Общие настройки

In [3]:
f_list = get_new_file_names("data/raw", "data/processed")
f_list

['Анкета_Денисов С.Н._24122023145530.xlsx',
 'Анкета_Макаров С.Н._18122023234522.xlsx']

## Лист 1

In [4]:
data_sheets = [pd.read_excel("data/raw/" + f_list[0], 
                             sheet_name=sheet_index) for sheet_index in range(4)]

In [5]:
data_sheets[0].columns = ["Вопрос", "Ответ"]
data_sheets[0] = data_sheets[0][2:].dropna(subset = ["Вопрос"]).reset_index(drop = True)
data_sheets[0]

,Вопрос,Ответ
0,1. Фамилия,Денисов
1,Имя,Семен
2,Отчество,Никитович
3,"2. Если изменяли фамилию, имя или отчество, то...","Фамилию, имя, отчество не изменял"
4,"3. Год, число, месяц и место рождения (село, д...","1966, 09 08, г. Владивосток"
5,"4. Паспорт (номер, серия, кем и когда выдан)","123456 78 90, паспортным столом № 1 г. Салават..."
6,"5. Имеете ли Вы заграничный паспорт (номер, се...","имею заграничный паспорт 6543217 89, ОУМС г. С..."
7,"6. Семейное положение (если вступали в брак, т...",Разведён. Вступал в брак с гражданкой Галушкин...
8,"7. Гражданство (если изменяли, то укажите, ког...",Гражданин Российской Федерации. Гражданин Респ...
9,8. Образование (когда и какие учебные заведени...,"Высшее, Башкирский политехнический институт, 1..."


### Тесты функций

Угадывание и исправление даты

In [6]:
date_guesser_corrector("1966, 09 08")

'1966, 08 Сентября'

## Лист 2

In [7]:
data_sheets[1].columns = ["Месяц и год поступления", 
                          "Месяц и год увольнения",
                          "Должность с указанием наименования организации",
                          "Адрес организации"]

data_sheets[1] = data_sheets[1][2:].dropna().reset_index(drop = True)
data_sheets[1]

,Месяц и год поступления,Месяц и год увольнения,Должность с указанием наименования организации,Адрес организации
0,09.1973,06.1983,средняя школа №15,"Башкирской АССР, г. Салават, ул. Советская, д. 5"
1,06.1983,11.09.1983,"находился на иждивении у родителей, проживал п...","Башкирской АССР, г. Салават, б-р Космонавтов, ..."
2,12.09.1983,22.09.1984,"студент, Уфимский филиал Московского нефтяного...","Башкирской АССР, г. Уфа, ул. Космонавтов, д. 1"
3,23.09.1984,09.1986,"военнослужащий срочной службы, в/ч 12345","Новосибирская обл., г. Тогучин"
4,10.1986,08.1987,"автослесарь, СТО № 2","Башкирской АССР, г. Салават, ул. Рабочая, д. 34"
5,09.1987,06.1991 г.,"студент, Уфимский филиал Московского нефтяного...","Башкирской АССР, г. Уфа, ул. Космонавтов, д. 1"
6,Июль 1991,10.2ф012,"инженер-технолог, главный инженер, ООО ""Обнинс...","Калужская обл., г. Обнинск, ул. Университетска..."
7,12.2012,по настоящее время,"менеджер по работе с клиентами, УЦБИ Маском","г. Москва, Старокалужское ш., д. 62, корп. 4, ..."


In [8]:
datevec = data_sheets[1]["Месяц и год поступления"].to_numpy()
datevec

array(['09.1973', '06.1983', '12.09.1983', '23.09.1984', '10.1986',
       '09.1987', 'Июль 1991', '12.2012'], dtype=object)

In [9]:
datevec = data_sheets[1]["Месяц и год увольнения"].to_numpy()
datevec

array(['06.1983', '11.09.1983', '22.09.1984', '09.1986', '08.1987',
       '06.1991 г.', '10.2ф012', 'по настоящее время'], dtype=object)

In [10]:
data_sheets[1]["Месяц и год поступления"].map(correct_date_condition2)

0    09.1973
1    06.1983
2    09.1983
3    09.1984
4    10.1986
5    09.1987
6    07.1991
7    12.2012
Name: Месяц и год поступления, dtype: object

In [11]:
data_sheets[1]["Месяц и год увольнения"].map(correct_date_condition2)

0               06.1983
1               09.1983
2               09.1984
3               09.1986
4               08.1987
5              06.1991.
6               10.2012
7    по настоящее время
Name: Месяц и год увольнения, dtype: object

## Лист 3

In [12]:
data_sheets[2].columns = ["Степень родства", 
                          "Фамилия, имя и отчество",
                          "Число, месяц, год и место рождения, гражданство",
                          "Место работы, должность",
                          "Адрес места жительства"]

data_sheets[2] = data_sheets[2][1:].dropna().reset_index(drop = True)
data_sheets[2]

,Степень родства,"Фамилия, имя и отчество","Число, месяц, год и место рождения, гражданство","Место работы, должность",Адрес места жительства
0,Отец,Макаров Николай Юрьевич,"25.01.1937, г. Стерлибашево Башкирской АССР, г...",Пенсионер,"Республика Башкортостан, г. Салават, ул. Калин..."
1,Мать,Макарова (Пурпурова) Валентина Михайловна,"07.06.1936, г. Стерлитамак Башкирской АССР, гр...",Умерла в 1995 г.,"Проживала по адресу: Республика Башкортостан, ..."
2,Brother,Макаров Андрей Николаевич,"07.04.1963, г. Салават Башкирской АССР, гражда...","Начальник производства, ООО ""АТЛАС РЕГИОН""","Московская обл., г. Дубна, ул. Боголюбова, д. ..."
3,Жена бывшая,Макарова (Галушкина) Ольга Владимировна,"11.01.1966, г. Воскресенск Московской обл., гр...","терапевт, городская поликлиника №2","Калужская обл., г. Обнинск, ул. Аксёнова, д. 3..."
4,Son,Макаров Роман Сергеевич,"14.10.1991, г. Воскресенск Московской обл., гр...","помощник руководителя направления, ООО ""Вентал""","Калужская обл., г. Обнинск, ул. Аксёнова, д. 3..."
5,Ex wife,Кобаева Виола Ролландовна,"05.08.1976, г. Душанбе Таджикской ССР, граждан...",сведениями не располагаю,"Калужская обл., г. Обнинск, ул. Гагарина, д. 5..."
6,Сын,Макаров Георгий Сергеевич,"06.01.2002, г. Обнинск Калужской обл., граждан...","студент, МИСИ","г. Москва, ул. Новочерёмушкинская, д. 27, кв. ..."
7,Жена бывшая,Окрошковская Елена Викторовна,"22.10.1972, г. Москва, гражданка РФ",сведениями не располагаю,"г. Москва, ул. Ярцевская, д. 29, корп.2, кв. 147"


In [15]:
def only_cyrillic(textstring: str) -> str:

    """
    Функция для проверки и корректировки по условиям 3: 
    1. Графа «Степень родства» пункта 15 должна содержать только буквы кириллицы
    2. В графе «Фамилия, имя и отчество» предыдущие фамилии (девичьи, изменённые) указываются в скобках. 
    Если у одного родственника несколько раз изменялась фамилия, то они указываются в скобках через запятую.

    Функция оставляет в строке только числа и точки. В случае, если указано "по настоящее время" - текст не меняется.

    Параметры:
    datestring : str
        Дата для форматирования

    Возвращает:
    string_out : str
        Строка, содержащая слово в желаемом падеже

    """

    return("".join(re.findall(pattern = "[а-яА-Я ]+", string = textstring)))

In [16]:
data_sheets[2]['Степень родства'].map(only_cyrillic)

0           Отец
1           Мать
2               
3    Жена бывшая
4               
5               
6            Сын
7    Жена бывшая
Name: Степень родства, dtype: object

## Лист 4

In [ ]:
data_sheets[3]

In [ ]:
pages_preprocessor('Анкета_Денисов С.Н._24122023145530.xlsx')